# Deploying a Keras or TensorFlow model using Amazon SageMaker

### Step 1. Set up

In [ ]:
import boto3, re
from sagemaker import get_execution_role

role = get_execution_role()

### Step 2. Load the Keras model using the json and weights file

Use the upload feature in jupyter to upload your model.json and model weights

If you saved your model in the TensorFlow ProtoBuf format, skip to "Step 4. Convert the TensorFlow model to an Amazon SageMaker-readable format.

In [ ]:
import tensorflow.keras
from tensorflow.keras.models import model_from_json

In [ ]:
json_file = open('model.json', 'r')
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)

In [ ]:
model.load_weights('model.h5')
print("Loaded model from disk")

### Step 3. Export the Keras model to the TensorFlow ProtoBuf format

In [ ]:
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants

In [ ]:
# Note: This directory structure will need to be followed
model_version = '1'
export_dir = 'export/Servo/' + model_version

In [ ]:
# Build the Protocol Buffer SavedModel at 'export_dir'
builder = builder.SavedModelBuilder(export_dir)

In [ ]:
# Create prediction signature to be used by TensorFlow Serving Predict API
signature = predict_signature_def(
    inputs={"inputs": model.input}, outputs={"score": model.output})

In [ ]:
from tensorflow.keras import backend as K

with K.get_session() as sess:
    # Save the meta graph and variables
    builder.add_meta_graph_and_variables(
        sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
    builder.save()

### Step 4. Convert TensorFlow model to a SageMaker readable format

Move the TensorFlow exported model into a directory export\Servo\. SageMaker will recognize this as a loadable TensorFlow model. Your directory and file structure should look like:

```
-export/
    |--Servo/
        |--{model version as an integer eg. 1}/
            |--variables/
            |--saved_model.pb
```

####  Tar the entire directory and upload to S3

In [ ]:
import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

In [ ]:
import sagemaker

session = sagemaker.Session()
bucket = session.default_bucket()
prefix = 'model'

inputs = session.upload_data(path='model.tar.gz', key_prefix=prefix)

### Step 5. Deploy the trained model

The entry_point file "train.py" can be an empty Python file. The requirement will be removed at a later date.

In [ ]:
!touch train.py

In [ ]:
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + bucket + '/model/model.tar.gz',
                                  role = role,
                                  framework_version = '1.12',
                                  entry_point = 'train.py')

In [ ]:
%%time
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge')

In [ ]:
from sagemaker.tensorflow.model import TensorFlowPredictor

predictor = TensorFlowPredictor(predictor.endpoint, session)

### Step 6. Invoke the endpoint

#### Invoke the SageMaker endpoint from the notebook

In [ ]:
import re
import json

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

with open('vocab_dict.json', 'r') as f:
    word_to_id = pickle.load(f)

def review_to_words(review):
    words = REPLACE_NO_SPACE.sub("", review.lower())
    words = REPLACE_WITH_SPACE.sub(" ", words)
    return words

def preprocess_input(text, vocab_dict, maxlen=100):
    review = review_to_words(text)
    tokens = review.split()
    int_tokens = [vocab_dict[token] for token in tokens]
    
    if len(int_tokens) >= maxlen:
        return int_tokens[:maxlen]
    else:
        diff = maxlen - len(int_tokens)
        zeros = [0 for i in range(diff)]
        return [zeros + int_tokens]

In [ ]:
text = "This movie was the Best Movie I have ever seen!"

In [ ]:
data = preprocess_input(text, word_to_id)

In [ ]:
predictor.predict(data)

#### Invoke the SageMaker endpoint using a boto3 client

In [ ]:
import json
import boto3
import numpy as np
import io
 
client = boto3.client('runtime.sagemaker')
# The sample model expects an input of shape [1,50]
data = np.random.randn(1, 50).tolist()
response = client.invoke_endpoint(EndpointName=endpoint_name, Body=json.dumps(data))
response_body = response['Body']
print(response_body.read())

### Step 7: Setting up AWS Lambda and API Gateway

The following code will allow lambda to invoke the endpoint

```python
import json
import re
import boto3

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

with open('vocab_dict.json', 'r') as fp:
    VOCAB_DICT = json.load(fp)


def review_to_words(review):
    words = REPLACE_NO_SPACE.sub("", review.lower())
    words = REPLACE_WITH_SPACE.sub(" ", words)
    return words

def preprocess_input(text, vocab_dict, maxlen=100):
    review = review_to_words(text)
    tokens = review.split()
    int_tokens = [vocab_dict[token] for token in tokens]
    
    if len(int_tokens) >= maxlen:
        return int_tokens[:maxlen]
    else:
        diff = maxlen - len(int_tokens)
        zeros = [0 for i in range(diff)]
        return [zeros + int_tokens]

def lambda_handler(event, context):
    
    data = preprocess_input(event['body'], VOCAB_DICT)

    # The SageMaker runtime is what allows us to invoke the endpoint that we've created.
    runtime = boto3.Session().client('sagemaker-runtime')

    # Now we use the SageMaker runtime to invoke our endpoint, sending the review we were given
    response = runtime.invoke_endpoint(EndpointName = 'sagemaker-tensorflow-2020-01-09-07-58-25-420',
                                       Body = json.dumps(data)) 

    # The response is an HTTP response whose body contains the result of our inference
    result = json.loads(response['Body'].read().decode("utf-8"))
    pred = result['outputs']['score']['floatVal'][0]

    return {
        'statusCode' : 200,
        'headers' : { 'Content-Type' : 'application/json', 'Access-Control-Allow-Origin' : '*', "Access-Control-Allow-Credentials" : True },
        'body' : pred
    }
    
```

Once you setup the lambda function, we need to connect Lambda to API Gateway

### Step 8. Clean up

To avoid incurring unnecessary charges, use the AWS Management Console to delete the resources that you created like the endpoint, S3 bucket, models, endpoint_configurations etc.